<a href="https://colab.research.google.com/github/ganaya-nih/NHLBI_Workshop_BigData092023/blob/main/AllofUs_SmokingPheWAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display, HTML
from datetime import date
import multiprocessing

import os
import statsmodels.api as sm

In [ ]:
my_bucket = os.getenv('WORKSPACE_BUCKET')
my_bucket

__Reading Phecode files__

In [ ]:
rollup_map=pd.read_csv("phecode_rollup_map.csv")
ICD9_Phecodes=pd.read_csv("ICDPhecodes.csv")
phecodes=pd.read_csv("ICDPhecodes.csv")
ICD9_exclude=pd.read_csv("ICD9PhecodeExclude.csv")
ICD9_IC10_Phecodes=pd.read_csv("phecode_map_icd9_10.csv")
# strip off extra column
ICD9_IC10_Phecodes = ICD9_IC10_Phecodes.iloc[:, 1:]
phecode_info=pd.read_csv("pheinfo.csv")
sex_at_birth_restriction = pd.read_csv("sex_at_birth_restriction[40].csv")

__<font color='red'>Warning</font>__
- Individual-level records data CANNOT be downloaded ( monitoring data transfer)
- In publications: results with numbers less than 20 should be suppressed

In [ ]:
phecodes_list=ICD9_IC10_Phecodes["phecode"].unique().tolist()
CDR_version=os.getenv("WORKSPACE_CDR")

In [ ]:
# Need to see which codes are in observation
query = ("""
SELECT distinct observation_source_value, vocabulary_id
FROM
    (SELECT DISTINCT person_id, observation_source_concept_id, observation_source_value, observation_date
        FROM `"""+ str(CDR_version) +""".observation`) AS obs
     INNER JOIN
        (SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id
            FROM `"""+str(CDR_version)+""".concept`
            where (vocabulary_id ='ICD9CM') or
            (vocabulary_id ='ICD10CM')) as concept
            on concept.concept_id = obs.observation_source_concept_id
""")
observation_codes = pd.read_gbq(query, dialect="standard")

In [ ]:
observation_icd9 = observation_codes[observation_codes.vocabulary_id=="ICD9CM"]["observation_source_value"]
observation_icd10 = observation_codes[observation_codes.vocabulary_id=="ICD10CM"]["observation_source_value"]

__Following function extracts the Phecodes from two EHR tables: Condition_occurrence and observation__

In [ ]:
def getPhecodeParticipants(phecodes_batch, return_dict, phecodes_list, CDR_version, num_processes):
    """
    Batching function for parallel extraction of participant Phecodes
    ======================================================================================================
    phecodes_batch: Pandas Dataframe of Phecodes
    return_dict:
    phecodes_list: List of phecodes to process
    CDR_version: String of current cdr version
    ## need to include rollup
    """
    size = int(np.ceil(len(phecodes_list)/num_processes))
    phecodes_code_list=phecodes_list[phecodes_batch*size:(phecodes_batch+1)*size]
    # changing for 4 cores
    #phecodes_code_list=phecodes_list[phecodes_batch*235:(phecodes_batch+1)*235]
    # for particular codes, there are rollups, so we need to count on the fly
    # note, there are icd9 codes that live in observation:
    phecodes=ICD9_IC10_Phecodes[ICD9_IC10_Phecodes["phecode"].isin(phecodes_code_list)]

    ## ICD Codes in condition_occurrence (if a code is in observation, then it shouldn't query anything here)
    icd9_codes_cond=phecodes[phecodes['vocabulary_id']=='ICD9CM']["code"].tolist()
    #
    icd9_codes_cond_str="'"+"','".join(icd9_codes_cond)+"'"

    icd10_codes_cond=phecodes[phecodes['vocabulary_id']=='ICD10CM']["code"].tolist()
    icd10_codes_cond_str="'"+"','".join(icd10_codes_cond)+"'"

    ## ICD Codes in observation

    icd9_codes_obs=np.intersect1d(observation_icd9,phecodes[phecodes['vocabulary_id']=='ICD9CM']["code"].tolist()).tolist()
    #
    icd9_codes_obs_str="'"+"','".join(icd9_codes_obs)+"'"

    icd10_codes_obs=np.intersect1d(observation_icd10,phecodes[phecodes['vocabulary_id']=='ICD10CM']["code"].tolist()).tolist()
    icd10_codes_obs_str="'"+"','".join(icd10_codes_obs)+"'"
        # there's a subtlety here that we need icd10 cm
    query="""SELECT DISTINCT icd.person_id,condition_start_date as start_date,condition_concept_id as cid,concept_code,vocabulary_id FROM `"""+CDR_version+""".concept`
    c  INNER JOIN `"""+CDR_version+""".condition_occurrence` icd  ON icd.condition_source_concept_id=c.concept_id
    WHERE vocabulary_id ='ICD9CM' AND  concept_code IN ("""+icd9_codes_cond_str+""")
    ORDER BY condition_start_date"""
    icdcodes1_cond=pd.read_gbq(query, dialect="standard")
    query="""SELECT DISTINCT icd.person_id,condition_start_date as start_date,condition_concept_id as cid,concept_code, vocabulary_id FROM `"""+CDR_version+""".concept`
    c  INNER JOIN `"""+CDR_version+""".condition_occurrence` icd  ON c.concept_id = icd.condition_source_concept_id
    WHERE vocabulary_id ='ICD10CM' AND  concept_code IN ("""+icd10_codes_cond_str+""")
    ORDER BY condition_start_date"""
    icdcodes2_cond=pd.read_gbq(query, dialect="standard")

    #Now observations

    query="""SELECT DISTINCT icd.person_id,observation_date as start_date, observation_concept_id as cid,concept_code,vocabulary_id FROM `"""+CDR_version+""".concept`
    c  INNER JOIN `"""+CDR_version+""".observation` icd  ON icd.observation_source_concept_id=c.concept_id
    WHERE vocabulary_id ='ICD9CM' AND  concept_code IN ("""+icd9_codes_obs_str+""")
    ORDER BY start_date"""
    icdcodes1_obs=pd.read_gbq(query, dialect="standard")
    query="""SELECT DISTINCT icd.person_id,observation_date as start_date,observation_concept_id as cid, concept_code, vocabulary_id FROM `"""+CDR_version+""".concept`
    c  INNER JOIN `"""+CDR_version+""".observation` icd  ON c.concept_id = icd.observation_source_concept_id
    WHERE vocabulary_id ='ICD10CM' AND  concept_code IN ("""+icd10_codes_obs_str+""")
    ORDER BY start_date"""

    icdcodes2_obs=pd.read_gbq(query, dialect="standard")

    icdcodes=pd.concat([icdcodes1_cond,icdcodes2_cond,icdcodes1_obs,icdcodes2_obs]).drop_duplicates() # drop duplicates within vocab, person id and date

    patients_phcode_count=icdcodes[["person_id","start_date","concept_code","vocabulary_id"]].drop_duplicates()[["person_id","concept_code","start_date","vocabulary_id"]]

    patients_phcode_count=pd.merge(phecodes[["code","phecode","vocabulary_id"]],patients_phcode_count,left_on=["code", "vocabulary_id"],right_on=["concept_code", "vocabulary_id"])

    return_dict[phecodes_batch]=patients_phcode_count

__The following code extract the count of Phecodes occurrence in participants records__

In [ ]:
def phecode_counts(num_processes, phecodes_list, CDR_version):
    size = int(np.ceil(len(phecodes_list)/num_processes))
    print("Processing phecodes in blocks of size: " + str(size))

    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    jobs = []
    for i in range(num_processes):
        p = multiprocessing.Process(target=getPhecodeParticipants, args=(i,return_dict, phecodes_list, CDR_version, num_processes)) #phecodes list needs to be included
        jobs.append(p)
        p.start()
    # close out and join jobs
    for proc in jobs:
        proc.join()
    # make into a dataframe
    phecodes_patients_list=pd.concat(list(return_dict.values()))

    # now merge to rollup map
    # Adjust to including the observation table

    phecodes_patients_list_merge = pd.merge(rollup_map, phecodes_patients_list,left_on = 'code', right_on = 'phecode')[["person_id","phecode_unrolled","start_date"]].drop_duplicates()
    ## clean a little bit more
    # get rid of any additional
    # Including the observation table
    phecodes_patients_counts_tmp=phecodes_patients_list_merge[["person_id","phecode_unrolled","start_date"]].groupby(["person_id","phecode_unrolled"],as_index=False).count()
    #
    ## so as not to break the other code
    phecodes_patients_counts_tmp.columns = ["person_id", "phecode", 'count']

    return(phecodes_patients_counts_tmp)

In [ ]:
phecodes_patients_counts = phecode_counts(num_processes=36, phecodes_list=phecodes_list, CDR_version = CDR_version)

__To save some credit and time, make sure to save the file__

In [ ]:
phecodes_patients_counts.to_csv("phecodes_patients_counts.csv")

__To run the PheWAS, we need covriaites that we will adjust to in the PheWAS model such as race, sex at birth, EHR length, etc__

In [ ]:
def make_covariates(df_indep_var,CDR_version):
    # first query from condition occurrence to get number of distinct codes and ehr length
    query="""SELECT person_id, min(condition_start_date) as min_cond_date,max(condition_start_date)as max_cond_date,COUNT(DISTINCT condition_concept_id) as cond_code_cnt
    FROM `"""+CDR_version+""".condition_occurrence` WHERE condition_start_date >='1980-01-01'
    GROUP BY person_id """

    ehr_covariate=pd.read_gbq(query, dialect="standard")
    print(ehr_covariate.shape[0],ehr_covariate[pd.isna(ehr_covariate["min_cond_date"])==True].shape[0],ehr_covariate[pd.isna(ehr_covariate["max_cond_date"])==True].shape[0] )

    # EHR Length is a bit subtle here. Observation date is a little tricky because birthdate is used in observation.
    # What we need to do is take the relevant codes from observation and find the minimum and maximum dates overall
    query = ("""
        SELECT distinct person_id, min(observation_date) as min_obs_date, max(observation_date) as max_obs_date, COUNT(DISTINCT observation_concept_id) as observation_code_cnt
        FROM
            (SELECT DISTINCT person_id, observation_source_concept_id, observation_source_value, observation_date, observation_concept_id
                FROM `"""+ str(CDR_version) +""".observation`) AS obs
             INNER JOIN
                (SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id
                    FROM `"""+str(CDR_version)+""".concept`
                    where (vocabulary_id ='ICD9CM') or
                    (vocabulary_id ='ICD10CM')) as concept
                    on concept.concept_id = obs.observation_source_concept_id
        group by person_id
        """)
    observation_code_dates = pd.read_gbq(query, dialect="standard")
    print(observation_code_dates.shape[0],observation_code_dates[pd.isna(observation_code_dates["min_obs_date"])==True].shape[0],observation_code_dates[pd.isna(observation_code_dates["max_obs_date"])==True].shape[0] )

    # note, this procedure takes into consideration NaTs, so if a person didn't have an observation code, it will
    # automatically take the minimum condition occurrence date.
    ehr_covariate_plus = pd.merge(observation_code_dates, ehr_covariate, on="person_id", how = 'outer')
    ehr_covariate_plus["min_obs_date"]=pd.to_datetime(ehr_covariate_plus["min_obs_date"])
    ehr_covariate_plus["min_cond_date"]=pd.to_datetime(ehr_covariate_plus["min_cond_date"])
    ehr_covariate_plus["max_obs_date"]=pd.to_datetime(ehr_covariate_plus["max_obs_date"])
    ehr_covariate_plus["max_cond_date"]=pd.to_datetime(ehr_covariate_plus["max_cond_date"])
    ehr_covariate_plus["min_date"] = ehr_covariate_plus[["min_obs_date","min_cond_date"]].min(axis =1,skipna=True)
    ehr_covariate_plus["max_date"] = ehr_covariate_plus[["max_obs_date","max_cond_date"]].max(axis =1,skipna=True)
    ehr_covariate_plus["code_cnt"] =  ehr_covariate_plus[["cond_code_cnt","observation_code_cnt"]].sum(axis =1)
    # for efficiency sake drop the extra columns
    ehr_covariate_plus = ehr_covariate_plus.drop(columns = ["min_obs_date","min_cond_date","max_obs_date","max_cond_date","cond_code_cnt","observation_code_cnt"])
    # ehr length defined as time between minimum of relevant observation dates
    print(ehr_covariate_plus['max_date']-ehr_covariate_plus['min_date'])
    print(ehr_covariate_plus.shape[0],ehr_covariate_plus[pd.isna(ehr_covariate_plus["min_date"])==True].shape[0],ehr_covariate_plus[pd.isna(ehr_covariate_plus["max_date"])==True].shape[0] )

    ehr_covariate_plus["ehr_length"]=(ehr_covariate_plus['max_date']-ehr_covariate_plus['min_date']).apply(lambda x:x.days)

    # Now Demographics

    query="""SELECT DISTINCT p.person_id,sex_at_birth_concept_id,gender_concept_id,race_concept_id,ethnicity_concept_id,year_of_birth, month_of_birth, day_of_birth FROM
    `"""+CDR_version+""".person` p
    """
    demo_patients=pd.read_gbq(query, dialect="standard")
    demo_patients["birthdate"] =pd.to_datetime(demo_patients["year_of_birth"].apply(str)+
                                               "-"+demo_patients["month_of_birth"].apply(str)+
                                               "-"+demo_patients["day_of_birth"].apply(str))
    demo_patients["age_today"] = (datetime.today()-demo_patients["birthdate"])/np.timedelta64(1,'Y')
    demo_patients=demo_patients[demo_patients["person_id"].isin(phecodes_patients_counts["person_id"])]

    # tidy up
    demo_patients_merge = demo_patients
    #demo_patients_merge=pd.merge(demo_patients, race_generalized,on="person_id")
    demo_patients_merge=pd.merge(demo_patients_merge, ehr_covariate_plus, on="person_id") # now merge to other ehr covariates # amended 04/17/2020
    demo_patients_merge["age_at_last_event"] = (demo_patients_merge["max_date"]-demo_patients_merge["birthdate"])/np.timedelta64(1,'Y')

    #############
    #race
    #############
    white = 8527
    black_aa = 8516
    asian = 8515
    # other
    more_than_one = 2000000008
    none_of_these = 45882607
    another_single_pop=2000000001
    other = [more_than_one, none_of_these,another_single_pop]
    #Unknown
    prefer_not_answer = 1177221
    skip = 903096
    no_match = 0
    unknown_race = [prefer_not_answer, skip, no_match]
    ## update indicators
    demo_patients_merge["white"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["AF"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["Asian"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["race_unk"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["Other"]=[0]*demo_patients_merge.shape[0]

    demo_patients_merge.loc[demo_patients_merge["race_concept_id"]==white,"white"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"]==black_aa,"AF"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"]==asian,"Asian"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"].isin(unknown_race),"race_unk"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"].isin(other),"Other"]=1


    #############
    #ethnicity
    #############
    #hisp/latino
    hisp_latino = 38003563
    # not hisp/latino
    not_hisp_lat = 38003564
    none_of_these = 45882607
    not_hisp = [not_hisp_lat, none_of_these]
    # unknown
    prefer_no_ans = 1177221
    skip= 903096
    unknown_eth = [prefer_no_ans, skip]
    # update indicators

    demo_patients_merge["hisp_lat"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["not_hisp_lat"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["unk_eth"]=[0]*demo_patients_merge.shape[0]
    # sex at birth
    demo_patients_merge.loc[ demo_patients_merge["ethnicity_concept_id"]==hisp_latino,"hisp_lat"]=1
    demo_patients_merge.loc[ demo_patients_merge["ethnicity_concept_id"]==not_hisp_lat,"not_hisp_lat"]=1
    demo_patients_merge.loc[(demo_patients_merge["ethnicity_concept_id"].isin(unknown_eth)),"unk_eth"]=1


    #############
    #sex at birth
    #############
    female = 45878463
    male = 45880669
    unknown = 2000000009 # prefer not to answer
    zer = 0
    ## update indicators
    demo_patients_merge["female"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["male"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["unk_sex"]=[0]*demo_patients_merge.shape[0]
    # sex at birth
    demo_patients_merge.loc[demo_patients_merge["sex_at_birth_concept_id"]==female,"female"]=1
    demo_patients_merge.loc[demo_patients_merge["sex_at_birth_concept_id"]==male,"male"]=1
    demo_patients_merge.loc[(demo_patients_merge["sex_at_birth_concept_id"]==unknown)|
                            (demo_patients_merge["sex_at_birth_concept_id"]==zer),"unk_sex"]=1
    # now merge to the indep var of interest
    demo_patients_cov=pd.merge(demo_patients_merge,df_indep_var, on = "person_id")
    demo_patients_cov=demo_patients_cov.drop_duplicates()
    return(demo_patients_cov)

__Defining PheWAS class that initializes variables, run Phewas and plot Manhattan plot__

In [ ]:
class PheWAS_Pool:
    """
    Class for performing PheWAS
    ======================================================================================================
    phecode_counts: Pandas Dataframe of Phecodes
    covariates: Pandas Dataframe of covariates to include in the analysis
    indep_var: String indicating the column in covariates that is the independent variable of interest
    CDR_version: String indicating CDR version
    phecode_process: list for phecodes to process
    min_cases: minimum number of cases for an individual phenotype to be analyzed
    cores: if not "", then specify number of cores to use in the analysis
    """
    def __init__(self, phecode_counts,
                 covariates,
                 indep_var_of_interest="",
                 CDR_version='R2019Q4R3',
                 phecode_process = 'all',
                 min_cases = 100,
                 independent_var_names=["AF","white","Asian","male","age_at_last_event",
                                        "ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
                 genderspec_independent_var_names=["AF","white","Asian","age_at_last_event","unk_sex","male",
                                                   "ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
                 show_res = False,
                 cores=""):
        print("~~~~~~~~~~~~~~~        Creating PheWAS AOU Object           ~~~~~~~~~~~~~~~~~~~~~")
        # create instance attributes
        self.indep_var_of_interest = indep_var_of_interest
        #update 09_5_2019: only process phecodes passed in phecode counts
        if phecode_process =='all':
            self.phecode_list = phecode_counts["phecode"].unique().tolist()
        else:
            self.phecode_list = phecode_process
        self.CDR_version = CDR_version
        self.cores = cores
        print("~~~~~~~~~~~~~~~       Merging Phecodes and Covariates       ~~~~~~~~~~~~~~~~~~~~~")
        self.demo_patients_phecodes = pd.merge(covariates,phecode_counts, on = "person_id")
        self.show_res = show_res
        self.independent_var_names = independent_var_names
        self.independent_var_names= list(np.append(np.array([self.indep_var_of_interest]),self.independent_var_names))
        self.genderspec_independent_var_names = genderspec_independent_var_names
        self.genderspec_independent_var_names= list(np.append(np.array([self.indep_var_of_interest]),self.genderspec_independent_var_names))
        self.remove_dup = list(np.append(np.array(["person_id"]),self.independent_var_names))
        self.min_cases = min_cases

    def runPheLogit(self, phecodes):
        # diagnostics
        for phecode in phecodes:
            ## init error
            error = "Other Error"
            try:
                # First, need to define the exclusions, sufficient to just use the ICD9 one since the overall phecodes
                # are the same
                phecode_exclusions=ICD9_exclude[ICD9_exclude["code"]==phecode]["exclusion_criteria"].unique().tolist()

                ## we need to do sex specific counting here.
                ## First find all people with at least 2 of the phecode
                cases=self.demo_patients_phecodes[(self.demo_patients_phecodes["phecode"]==phecode)&
                                                  (self.demo_patients_phecodes["count"]>=2)]

                ## now determine if there is a sex specific restriction
                ## this is convoluted, but it is written this way to avoid storing another copy of all the data in memory
                male_only = sex_at_birth_restriction[sex_at_birth_restriction.phecode.astype(float)==phecode]["male_only"]==True
                female_only = sex_at_birth_restriction[sex_at_birth_restriction.phecode.astype(float)==phecode]["female_only"]==True

                # now, if sex at birth specific, filter all the people to the proper sex and set the right covariates
                if np.array(male_only)[0]==True :
                    analysis_independent_var_names=self.genderspec_independent_var_names
                    cases=cases[cases['male']==1]#restrict to males

                # if female only, then restrict to people who are female == 1
                elif np.array(female_only)[0]==True:

                    analysis_independent_var_names=self.genderspec_independent_var_names
                    cases=cases[cases['female']==1]#restrict to females

                # otherwise there is no restriction, so we can use sex at birth in the analyses
                else:
                    analysis_independent_var_names=self.independent_var_names

                # Now cases have been properly modified and so we remove any duplicates and restrict the analysis to just the regressors
                cases=cases[self.remove_dup].drop_duplicates()[analysis_independent_var_names]

                # Now test to see if we have enough cases
                # This is written like this to avoid unnecessary compute for phecodes
                # for which we don't have enough cases

                if cases.shape[0]>=self.min_cases:
                    # if it passes
                    # create set all people that need to be excluded
                    exclude=self.demo_patients_phecodes[(self.demo_patients_phecodes["phecode"].isin(np.append(phecode_exclusions,phecode)))]#&(control["concept_code"]<=2)]

                    ## if sex specific, restrict analyses to just that sex at birth
                    if np.array(male_only)[0]==True:
                        control=self.demo_patients_phecodes[(self.demo_patients_phecodes.person_id.isin(exclude.person_id)==False)&
                                                            self.demo_patients_phecodes.male==1] # pick off just the males

                    # if female only, then restrict to people who are female == 1
                    elif np.array(female_only)[0]==True:
                        control=self.demo_patients_phecodes[(self.demo_patients_phecodes.person_id.isin(exclude.person_id)==False)&
                                                            (self.demo_patients_phecodes.female==1)] # pick off just the males

                    # otherwise there is no restriction, so we can use sex at birth in the analyses
                    else:
                        control=self.demo_patients_phecodes[self.demo_patients_phecodes.person_id.isin(exclude.person_id)==False]

                    # Now controls have been properly modified and so we remove any duplicates and restrict the analysis to just the regressors
                    control=control[self.remove_dup].drop_duplicates()[analysis_independent_var_names]
                    ############################################################################################
                    ## Perform Logistic regression
                    ## Now run through the logit function from stats models
                    ############################################################################################
                    y=[1]*cases.shape[0]+[0]*control.shape[0]
                    regressors=pd.concat([cases,control])
                    regressors=sm.tools.add_constant(regressors)
                    logit = sm.Logit(y, regressors, missing = 'drop')
                    result = logit.fit(disp=False)

                    # choose to see results on the fly
                    if self.show_res == True:
                        print(result.summary())
                    else:
                        pass
                    # return
                    results_as_html = result.summary().tables[0].as_html()
                    converged = pd.read_html(results_as_html)[0].iloc[5,1]
                    # now other quants
                    results_as_html = result.summary().tables[1].as_html()
                    res = pd.read_html(results_as_html, header = 0, index_col=0)[0]
                    p_value =  result.pvalues[self.indep_var_of_interest]
                    beta_ind = result.params[self.indep_var_of_interest]
                    conf_int_1=res.loc[self.indep_var_of_interest]['[0.025']
                    conf_int_2=res.loc[self.indep_var_of_interest]['0.975]']

                    self.return_dict[phecode] =[phecode,cases.shape[0],control.shape[0],p_value,
                                                beta_ind, conf_int_1, conf_int_2, converged]
                else:
                    error = "Error in Phecode: "+str(phecode)+ ": Number of cases less than minimum of "+str(self.min_cases)
                del [control, cases, regressors]
            except:
                print(error)
    # now define function for running the phewas
    def run(self):
        # we will use multiprocessing tool for this
        #update_09_04_2019
        manager = multiprocessing.Manager()
        self.return_dict = manager.dict()
        ## this needs to have a particular list structure for the multiprocessing
        partitions = [list(ind) for ind in np.array_split(self.phecode_list, self.cores)] # just check for benchmark
        pool = multiprocessing.Pool(processes=self.cores)
        map_result = pool.map_async(self.runPheLogit, partitions) #what this is doing is iterating over the list of indices I made before
        # and subsetting the phenotypes to just the batch in index i of indices
        pool.close()
        pool.join()

        logit_Phecode_results=[self.return_dict[k] for k in self.return_dict.keys()]
        logit_Phecode_results=pd.DataFrame(logit_Phecode_results)
        logit_Phecode_results.columns=["phecode","cases","control","p_value","beta_ind","conf_int_1", "conf_int_2", "converged"]

        logit_Phecode_results["code_val"] = logit_Phecode_results["phecode"]
        logit_Phecode_results["neg_p_log_10"] = -np.log10(logit_Phecode_results["p_value"])
        logit_Phecode_results=pd.merge(phecode_info,logit_Phecode_results)
        # now save logit phecode as attribute
        self.logit_Phecode_results = logit_Phecode_results

    def Manhattan_Plot_Plus(self, group = "all"):
        """
        Method for plotting Manhattan Plot
        ======================================================================================================
        group: list of groups to display (e.g. neoplasms)
        """
        PheWAS_results_ehr =self.logit_Phecode_results

        PheWAS_results_ehr.loc[PheWAS_results_ehr["color"]=="darkorange1","color"]="orange"
        PheWAS_results_ehr.loc[PheWAS_results_ehr["color"]=="darkseagreen4","color"]="darkgreen"
        PheWAS_results_ehr.loc[PheWAS_results_ehr["color"]=="coral4","color"]="coral"
        PheWAS_results_ehr.loc[PheWAS_results_ehr["color"]=="chartreuse4","color"]="chartreuse"
        PheWAS_results_ehr.loc[PheWAS_results_ehr["color"]=="royalblue4","color"]="royalblue"
        PheWAS_results_ehr.loc[PheWAS_results_ehr["color"]=="gray50","color"]="gray"
        x2,y2 = 20,50
        # subset to particular group

        if group != "all":
            PheWAS_results_ehr = PheWAS_results_ehr[PheWAS_results_ehr["group"]==group]

        fig, ax = plt.subplots(figsize=(15,8))
        benf_corr=.05/phecodes.PheCode.unique().shape[0]
        pos_beta=PheWAS_results_ehr[PheWAS_results_ehr["beta_ind"]>=0]
        neg_beta=PheWAS_results_ehr[PheWAS_results_ehr["beta_ind"]<0]

        ax.scatter(pos_beta["code_val"], pos_beta["neg_p_log_10"], c=pos_beta['color'],marker='^')
        ax.scatter(neg_beta["code_val"], neg_beta["neg_p_log_10"], c=neg_beta['color'],marker='v')
        ax.hlines(-np.log10(.05),0,PheWAS_results_ehr["code_val"].max()+1,colors='r',label = "0.05")
        ax.hlines(-np.log10(benf_corr),0,PheWAS_results_ehr["code_val"].max()+1,colors='g',label = "Bonferroni Threshold (0.05/1847)")
        PheWas_ticks=PheWAS_results_ehr[["phecode","group"]].groupby("group",as_index=False).mean()

        # reshape the final plot to just fit the phecodes in the subgroup
        plt.xlim(PheWAS_results_ehr["phecode"].min(),PheWAS_results_ehr["phecode"].max())
        plt.xticks(PheWas_ticks["phecode"],PheWas_ticks["group"],rotation=45,ha="right")
        pos_beta_top=pos_beta[pos_beta["p_value"]<benf_corr].sort_values("neg_p_log_10",ascending=False).iloc[:15,][["code_val","neg_p_log_10","description"]]
        #Drop infs
        #
        pos_beta_top = pos_beta_top[~np.isinf(pos_beta_top["neg_p_log_10"])]
        neg_beta_top=neg_beta[neg_beta["p_value"]<benf_corr].sort_values("neg_p_log_10",ascending=False).iloc[:10,][["code_val","neg_p_log_10","description"]]
        ## drop infs
        neg_beta_top= neg_beta_top[~np.isinf(neg_beta_top["neg_p_log_10"])]

        for i,row in pos_beta_top.iterrows():
            ax.annotate(row["description"], (row["code_val"], row["neg_p_log_10"]),xytext=(x2, y2), textcoords='offset points',
            arrowprops=dict(arrowstyle="->", color='black'))
        for i,row in neg_beta_top.iterrows():
            ax.annotate(row["description"], (row["code_val"], row["neg_p_log_10"]),xytext=(x2, y2), textcoords='offset points',
            arrowprops=dict(arrowstyle="->", color='black'))
        # assign top pos and neg to self
        self.pos_beta_top = pos_beta_top
        self.neg_beta_top = neg_beta_top
        from matplotlib.lines import Line2D
        # add legend elements
        legend_elements =  [Line2D([0], [0], color='g', lw=4, label='Bonferroni Correction'),
                            Line2D([0], [0], color='r', lw=4, label='Nominal Significance Level'),
                            Line2D([0], [0], marker='v', label='Protective Effect',
                                  markerfacecolor='b', markersize=15),
                           Line2D([0], [0], marker='^', label='Non-Protective Effect',
                                  markerfacecolor='b', markersize=15),]
        ax.legend(handles=legend_elements, bbox_to_anchor =(0.5,-0.27), loc='lower center')
        ax.set_ylabel(r'$-\log_{10}$(p-value)')

__Extracting smoking status from EHR__

In [ ]:
query = ("""
SELECT distinct person_id,  count(distinct code_date) as count
FROM
    (SELECT DISTINCT person_id, code_date
    from
    (SELECT DISTINCT person_id, observation_source_concept_id, observation_source_value, observation_date as code_date
        FROM `"""+ str(CDR_version) +""".observation`) AS obs
     INNER JOIN
        (SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id
            FROM `"""+str(CDR_version)+""".concept`x
            where (concept_code = 'V15.82'
            and vocabulary_id ='ICD9CM') or
            (concept_code in('Z72.0','Z87.891','Z71.6')
            and vocabulary_id ='ICD10CM')) as concept
            on concept.concept_id = obs.observation_source_concept_id
    union all
    (select DISTINCT person_id, code_date
    from
     (SELECT DISTINCT person_id, condition_source_concept_id, condition_source_value, condition_start_date as code_date
        FROM `"""+ str(CDR_version) +""".condition_occurrence`) AS cond
     INNER JOIN
        (
        SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id
            FROM `"""+str(CDR_version)+""".concept`
            where (concept_code like '305.1%' or concept_code = 'V15.82' or concept_code like '649.0%'
            or concept_code like "989.84%"
            and vocabulary_id ='ICD9CM') or
            (concept_code in('Z72.0') or concept_code like 'O99.33%' or concept_code like "F17.2%"
                or concept_code like 'T65.2%'
            and vocabulary_id ='ICD10CM')) as concept
            on concept.concept_id = cond.condition_source_concept_id
            )  )
group by person_id
""")
df_smoke_controls_exclusion = pd.read_gbq(query, dialect="standard")
df_smoke_controls_exclusion.to_csv("df_smoke_controls_exclusion.csv")

In [ ]:
query = ("""
SELECT distinct person_id,  count(distinct code_date) as count
FROM
    (
    (SELECT DISTINCT person_id, code_date
    from
    (SELECT DISTINCT person_id, observation_source_concept_id, observation_source_value, observation_date as code_date
        FROM `"""+ str(CDR_version) +""".observation`) AS obs
     INNER JOIN
        (SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id
            FROM `"""+str(CDR_version)+""".concept`
            where (concept_code = 'V15.82'
            and vocabulary_id ='ICD9CM') or
            (concept_code in('Z72.0','Z87.891','Z71.6')
            and vocabulary_id ='ICD10CM')) as concept
            on concept.concept_id = obs.observation_source_concept_id)
    union all
    (select DISTINCT person_id, code_date
    from
     (SELECT DISTINCT person_id, condition_source_concept_id, condition_source_value, condition_start_date as code_date
        FROM `"""+ str(CDR_version) +""".condition_occurrence`) AS cond
     INNER JOIN
        (
        SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id
            FROM `"""+str(CDR_version)+""".concept`
            where (concept_code like '305.1%' or concept_code = 'V15.82' or concept_code like '649.0%'
            or concept_code like "989.84%"
            and vocabulary_id ='ICD9CM') or
            (concept_code in('Z72.0') or concept_code like 'O99.33%'
                or (concept_code like 'F17.2%' and concept_code not like 'F17.22%')
                or (concept_code like 'T65.2%' and concept_code not like 'T65.21%')
            and vocabulary_id ='ICD10CM')) as concept
            on concept.concept_id = cond.condition_source_concept_id
            )  )
group by person_id
""")
df_smoke_cases_inclusion = pd.read_gbq(query, dialect="standard")
df_smoke_cases_inclusion.to_csv("df_smoke_cases_inclusion.csv")

In [ ]:
df_smoke_cases_inclusion["ever_smoke_ehr"] = (df_smoke_cases_inclusion["count"]>=2)+0
# now -9 out the folks with one code
df_smoke_cases_inclusion["ever_smoke_ehr"] = df_smoke_cases_inclusion["ever_smoke_ehr"].map({1:1,0:-9})
df_smoke_cases_inclusion.ever_smoke_ehr

In [ ]:
df_smoke_cases_inclusion.ever_smoke_ehr.value_counts()

In [ ]:
# now find everyone without any smoking
# amend to include people with icd9cm and icd10 cm in observation
# use df_smoke_controls_exclusion person_ids since we don't want anyone with chewing tobacco in controls
query = ("""
select distinct person_id from `"""+ str(CDR_version)+""".condition_occurrence`
where person_id not in """+"("+' '.join([str(id)+"," for id in np.unique(df_smoke_controls_exclusion.person_id)])[:-1]+")"+""";
""")
smoke_nos = pd.read_gbq(query, dialect="standard")
# people with at least one icd9/10cm code in observation but no smoking code (including chewing tobacco)
# so use df_smoke_controls_exclusion person_ids
# deduplicate the person_ids
query = ("""
SELECT distinct person_id
FROM
    (SELECT DISTINCT person_id, observation_source_concept_id, observation_source_value, observation_date
        FROM `"""+ str(CDR_version) +""".observation`) AS obs
     INNER JOIN
        (SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id
            FROM `"""+str(CDR_version)+""".concept`
            where (vocabulary_id ='ICD9CM') or
            (vocabulary_id ='ICD10CM')) as concept
            on concept.concept_id = obs.observation_source_concept_id
       where person_id not in """+"("+' '.join([str(id)+"," for id in np.unique(df_smoke_controls_exclusion.person_id)])[:-1]+")"+"""
""")
observation_coded_participants_no_smoke = pd.read_gbq(query, dialect="standard")

# append to smoke_nos from condition occurrence
smoke_nos = smoke_nos.append(observation_coded_participants_no_smoke)
# deduplicate
smoke_nos = smoke_nos.drop_duplicates()
smoke_nos["ever_smoke_ehr"] = 0
smoke_nos.to_csv("smoke_nos.csv")
# and remove folks with only one code for smoke
df_smoke_ehr = df_smoke_cases_inclusion[['person_id','ever_smoke_ehr']]

df_smoke_ehr = df_smoke_ehr.append(smoke_nos)
df_smoke_ehr.head() # this is what we will use in our analysis
df_smoke_ehr.to_csv("df_smoke_ehr.csv")


In [ ]:
df_smoke_ehr.ever_smoke_ehr.value_counts()

__Extract covariats for participants with EHR smoking status__

In [ ]:
# subset out people with 1 code in covariates, recall that they were coded as -9
demo_patients_cov = make_covariates(df_smoke_ehr[df_smoke_ehr['ever_smoke_ehr']!=-9], CDR_version = CDR_version)

__Extract participants who have any EHR data for inclusion in the analysis__

In [ ]:
# now cross-reference with ehr folks

query = """
WITH ehr AS (
Select
   distinct person_id
from `{dataset}.measurement` as m
left join `{dataset}.measurement_ext` as mm on m.measurement_id = mm.measurement_id
where lower(mm.src_id) like 'ehr site%'

union distinct

Select
   distinct person_id
from `{dataset}.condition_occurrence` as m
left join `{dataset}.condition_occurrence_ext` as mm on m.condition_occurrence_id = mm.condition_occurrence_id
where lower(mm.src_id) like 'ehr site%'

union distinct

Select
   distinct person_id
from `{dataset}.device_exposure` as m
left join `{dataset}.device_exposure_ext` as mm on m.device_exposure_id = mm.device_exposure_id
where lower(mm.src_id) like 'ehr site%'

union distinct

Select
   distinct person_id
from `{dataset}.drug_exposure` as m
left join `{dataset}.drug_exposure_ext` as mm on m.drug_exposure_id = mm.drug_exposure_id
where lower(mm.src_id) like 'ehr site%'

union distinct

Select
   distinct person_id
from `{dataset}.observation` as m
left join `{dataset}.observation_ext` as mm on m.observation_id = mm.observation_id
where lower(mm.src_id) like 'ehr site%'

union distinct

Select
   distinct person_id
from `{dataset}.procedure_occurrence` as m
left join `{dataset}.procedure_occurrence_ext` as mm on m.procedure_occurrence_id = mm.procedure_occurrence_id
where lower(mm.src_id) like 'ehr site%'

union distinct

Select
   distinct person_id
from `{dataset}.visit_occurrence` as m
left join `{dataset}.visit_occurrence_ext` as mm on m.visit_occurrence_id = mm.visit_occurrence_id
where lower(mm.src_id) like 'ehr site%'
)

select distinct
"EHR" as data,
person_id
FROM ehr
"""

ehr_df = pd.read_gbq(query.format(dataset=CDR_version), dialect='standard')
ehr_df.to_csv("ehr_df.csv")

__Run the following cell only if you want to read the above files after saving them__

In [ ]:
df_smoke_ehr=pd.read_csv("df_smoke_ehr.csv")
df_smoke_cases_inclusion=pd.read_csv("df_smoke_cases_inclusion.csv")
df_smoke_cases_inclusion=pd.read_csv("df_smoke_controls_exclusion.csv")


__Extracting infomration about smoking from surveys__

In [ ]:
## Check out w.ppi smoking
# inner join the question value_as_concept_id to the concept_id of the response
#1585857
query =("""
select distinct qs.person_id, conc.concept_name from
(SELECT DISTINCT person_id, observation_source_value, observation_source_concept_id, value_source_concept_id
    FROM `"""+ str(CDR_version)+""".observation` where observation_source_concept_id =1585857) as qs
INNER JOIN
(SELECT concept_id, concept_name from `"""+ str(CDR_version)+""".concept`) as conc
on conc.concept_id = qs.value_source_concept_id
""")
obs_q = pd.read_gbq(query, dialect="standard")
obs_q.concept_name.value_counts()


## We now exract the person id and concept name (responses)
PPI_ever_smoke = obs_q[["person_id", "concept_name"]]

# Subset to where we have smoking info by merging with PPI smoke
# We just want people who responded either yes or no - since skip/prefer not to answer is not very informative
PPI_smoke_Y_N = PPI_ever_smoke[(PPI_ever_smoke["concept_name"]=="100 Cigs Lifetime: No")|(PPI_ever_smoke["concept_name"]=="100 Cigs Lifetime: Yes")]

PPI_smoke_Y_N["ever_smoke_ppi"] = (PPI_smoke_Y_N["concept_name"]=="100 Cigs Lifetime: Yes")+0

In [ ]:
demo_patients_cov_ppi = make_covariates(PPI_smoke_Y_N[["person_id","ever_smoke_ppi"]], CDR_version = CDR_version)


In [ ]:
demo_patients_cov_ppi.to_csv("demo_patients_cov_ppi.csv")

In [ ]:
PPI_ever_smoke.groupby("concept_name",as_index=False).count()

__Reading the files that will be used in the PheWAS analysis__


In [ ]:
phecodes_patients_counts=pd.read_csv("phecodes_patients_counts.csv")
demo_patients_cov=pd.read_csv("demo_patients_cov.csv")
demo_patients_cov_ppi=pd.read_csv("demo_patients_cov_ppi.csv")

In [ ]:
demo_patients_cov.shape,demo_patients_cov_ppi.shape

In [ ]:
print("Number of smokers using EHR",demo_patients_cov["ever_smoke_ehr"].sum())
print("Number of smokers using surveys",demo_patients_cov_ppi["ever_smoke_ppi"].sum())
ehr_smokes=demo_patients_cov[demo_patients_cov["ever_smoke_ehr"]==1].person_id.tolist()
ppi_smokes=demo_patients_cov_ppi[demo_patients_cov_ppi["ever_smoke_ppi"]==1].person_id.tolist()
print("Overlap between EHR and survey",len(set(ehr_smokes).intersection(ppi_smokes)))
print("Smoker only in EHR",len(set(ehr_smokes).difference(ppi_smokes)))
print("Smoker only in Survey",len(set( ppi_smokes).difference(ehr_smokes)))


__Demographics for participants who will be inlcuded in Smkoing EHR analysis__

In [ ]:
print("Demographics of participants who will be inlcuded in Smkoing EHR analysis")
demo_patients_cov.shape[0],demo_patients_cov["white"].sum(),demo_patients_cov["AF"].sum(),demo_patients_cov["Asian"].sum(),demo_patients_cov["race_unk"].sum(),demo_patients_cov["Other"].sum()
demo_ehr_race=pd.DataFrame(demo_patients_cov[["white","AF","Asian","race_unk","Other"]].sum()).reset_index()
demo_ehr_race.columns=["Race","Count"]
demo_ehr_race["Perc"]=100*demo_ehr_race["Count"]/demo_patients_cov.shape[0]
demo_ehr_race

In [ ]:

print("Sex at Birth of participants who will be inlcuded in Smkoing EHR analysis")
demo_ehr_sex=pd.DataFrame(demo_patients_cov[['female', 'male', 'unk_sex']].sum()).reset_index()
demo_ehr_sex.columns=["Race","Count"]
demo_ehr_sex["Perc"]=100*demo_ehr_sex["Count"]/demo_patients_cov.shape[0]
demo_ehr_sex


__Demographics for participants who will be inlcuded in Surevy Smkoing analysis__

In [ ]:
print("Race of participants who will be inlcuded in Smkoing survey analysis")
demo_ppi_race=pd.DataFrame(demo_patients_cov_ppi[["white","AF","Asian","race_unk","Other"]].sum()).reset_index()
demo_ppi_race.columns=["Race","Count"]
demo_ppi_race["Perc"]=100*demo_ppi_race["Count"]/demo_patients_cov_ppi.shape[0]
demo_ppi_race



In [ ]:
print("Sex at birth of participants who will be inlcuded in Smkoing survey analysis")
demo_ppi_sex=pd.DataFrame(demo_patients_cov_ppi[['female', 'male', 'unk_sex']].sum()).reset_index()
demo_ppi_sex.columns=["Race","Count"]
demo_ppi_sex["Perc"]=100*demo_ppi_sex["Count"]/demo_patients_cov_ppi.shape[0]
demo_ppi_sex


In [ ]:
print("Phecode groups")
phecode_info["group"].unique().tolist()

__Running analysis on respiratory phecodes__

__EHR smoking PheWAS analysis__

In [ ]:
respiratory=phecode_info[phecode_info["group"]=="respiratory"]["phecode"].unique().tolist()
# Instantiate an object of class PheWAS
test_pool = PheWAS_Pool(phecode_counts = phecodes_patients_counts,
              covariates= demo_patients_cov, CDR_version= CDR_version,
             independent_var_names=["AF","white","Asian","male","age_at_last_event",
                                    "ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
            genderspec_independent_var_names=["AF","white","Asian","age_at_last_event",
                                              "ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
              indep_var_of_interest= "ever_smoke_ehr",show_res = True,
            phecode_process= np.array(respiratory),
              cores =16,
            #independent_var_names=["male", "ever_smoke_ehr"]
        )

In [ ]:
test_pool.run()
test_pool.logit_Phecode_results

In [ ]:


# This snippet assumes you run setup first

# This code saves your PheWAS results into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = test_pool.logit_Phecode_results

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'phewas_smoking_ehr_9_2023_demo_respiratory_project.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
os.system(f"gsutil cp './{destination_filename}' '{my_bucket}/data/'")
print(f'[INFO] {destination_filename} is successfully uploaded in your bucket.')

__Survey smoking PheWAS analysis__

__Running the PheWAS on two sections since the dataset will be big and your kernel might die__

In [ ]:
respiratory=phecode_info[phecode_info["group"]=="respiratory"]["phecode"].unique().tolist()[:40]
# Instantiate an object of class PheWAS
Phewas_part1 = PheWAS_Pool(phecode_counts = phecodes_patients_counts,
              covariates= demo_patients_cov_ppi, CDR_version= CDR_version,
             independent_var_names=["AF","white","Asian","male","age_at_last_event",
                                    "ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
            genderspec_independent_var_names=["AF","white","Asian","age_at_last_event",
                                              "ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
              indep_var_of_interest= "ever_smoke_ppi",show_res = True,
            phecode_process= np.array(respiratory),
              cores =16,
            #independent_var_names=["male", "ever_smoke_ehr"]
        )

In [ ]:
Phewas_part1.run()
Phewas_part1.logit_Phecode_results

In [ ]:


# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = Phewas_part1.logit_Phecode_results

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'phewas_smoking_ppi_9_2023_demo_respiratory_project_part2.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
os.system(f"gsutil cp './{destination_filename}' '{my_bucket}/data/'")
print(f'[INFO] {destination_filename} is successfully uploaded in your bucket.')

In [ ]:
respiratory=phecode_info[phecode_info["group"]=="respiratory"]["phecode"].unique().tolist()[40:]
# Instantiate an object of class PheWAS
Phewas_part2 = PheWAS_Pool(phecode_counts = phecodes_patients_counts,
              covariates= demo_patients_cov_ppi, CDR_version= CDR_version,
             independent_var_names=["AF","white","Asian","male","age_at_last_event",
                                    "ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
            genderspec_independent_var_names=["AF","white","Asian","age_at_last_event",
                                              "ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
              indep_var_of_interest= "ever_smoke_ppi",show_res = True,
            phecode_process= np.array(respiratory),
              cores =16,
            #independent_var_names=["male", "ever_smoke_ehr"]
        )

In [ ]:
Phewas_part2.run()
Phewas_part2.logit_Phecode_results

In [ ]:


# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = Phewas_part2.logit_Phecode_results

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'phewas_smoking_ppi_9_2023_demo_respiratory_project_part2.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
os.system(f"gsutil cp './{destination_filename}' '{my_bucket}/data/'")
print(f'[INFO] {destination_filename} is successfully uploaded in your bucket.')

__You can read the files later for analysis without needing to run the code__

In [ ]:
#plot manhaten plot
phewas_smoking_ehr_9_2023_demo_respiratory_project=pd.read_csv("phewas_smoking_ehr_9_2023_demo_respiratory_project.csv")
phewas_smoking_ehr_9_2023_demo_respiratory_project


In [ ]:
phewas_smoking_ehr_9_2023_demo_respiratory_project.head()

In [ ]:
respiratory=phecode_info[phecode_info["group"]=="respiratory"]["phecode"].unique().tolist()[40:]
# Instantiate an object of class PheWAS
test_pool = PheWAS_Pool(phecode_counts = phecodes_patients_counts,
              covariates= demo_patients_cov_ppi, CDR_version= CDR_version,
             independent_var_names=["AF","white","Asian","male","age_at_last_event",
                                    "ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
            genderspec_independent_var_names=["AF","white","Asian","age_at_last_event",
                                              "ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
              indep_var_of_interest= "ever_smoke_ppi",show_res = True,
            phecode_process= np.array(respiratory),
              cores =16,
            #independent_var_names=["male", "ever_smoke_ehr"]
        )
test_pool.logit_Phecode_results=phewas_smoking_ehr_9_2023_demo_respiratory_project
test_pool.Manhattan_Plot_Plus()

In [ ]:
phewas_smoking_ppi_9_2023_demo_respiratory_project_part1=pd.read_csv("phewas_smoking_ppi_9_2023_demo_respiratory_project_part1.csv")
phewas_smoking_ppi_9_2023_demo_respiratory_project_part2=pd.read_csv("phewas_smoking_ppi_9_2023_demo_respiratory_project_part2.csv")
phewas_smoking_ppi_9_2023_demo_respiratory_project=pd.concat([phewas_smoking_ppi_9_2023_demo_respiratory_project_part1,phewas_smoking_ppi_9_2023_demo_respiratory_project_part2])

respiratory=phecode_info[phecode_info["group"]=="respiratory"]["phecode"].unique().tolist()[40:]
# Instantiate an object of class PheWAS
test_pool = PheWAS_Pool(phecode_counts = phecodes_patients_counts,
              covariates= demo_patients_cov_ppi, CDR_version= CDR_version,
             independent_var_names=["AF","white","Asian","male","age_at_last_event",
                                    "ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
            genderspec_independent_var_names=["AF","white","Asian","age_at_last_event",
                                              "ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
              indep_var_of_interest= "ever_smoke_ppi",show_res = True,
            phecode_process= np.array(respiratory),
              cores =16,
            #independent_var_names=["male", "ever_smoke_ehr"]
        )
test_pool.logit_Phecode_results=phewas_smoking_ppi_9_2023_demo_respiratory_project
test_pool.Manhattan_Plot_Plus()

